# IBM Applied Data Science Capstone Project

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Assignment 1: Setup notebook</a>

2. <a href="#item2">Assignment 2: Scrape wiki and generate map with clusters</a> 
    
3. <a href="#item3">Assignment 3: Final assignment</a> 
    
</font>
</div>

<a id='item1'></a>
# Assignment 1
Notebook setup and post to Github

In [ ]:
# Initial setup of libraries
import pandas as pd
import numpy as np

In [ ]:
print('Hello Capstone Project Course!')

<a id='item2'></a>
# Assignment 2
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto  
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.  
Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.  


## 2.1. Wrangle data: Scrape wiki article
Ref: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  

In [ ]:
from bs4 import BeautifulSoup
import requests

Scrape the wiki article:

In [ ]:
#URL to scrape
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

# Verify the scrape
print("Page scraped! Page Title:",soup.title.string)

Create dataframe:

In [ ]:
table = soup.find("table", attrs={"class": "wikitable sortable"})
table_data = table.find_all("tr")

In [ ]:
df = []
for tr in table_data:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    df.append(row)

df = pd.DataFrame(df, columns=["Postal Code", "Borough", "Neighbourhood"])
df

Cleanup:

In [ ]:
df.drop(index=0, inplace=True)
df

Checked below is to see which rows 'Neighbourhood'=='Borough' as per instructor comment:  
>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  

In [ ]:
assigned=df[df['Borough']==df['Neighbourhood']]
assigned.groupby('Borough').count()

After check, all 'Borough' == 'Neighbourhood' have a value of 'Not assigned'. Will proceed to drop rows with 'Not assigned.

In [ ]:
df = df[~df.Borough.str.contains("Not assigned")]
df = df[~df.Neighbourhood.str.contains("Not assigned")]
df

In [ ]:
df.reset_index(inplace=True, drop=True)
df

In [ ]:
df.shape

## 2.2. Get geo coordinates for postal codes
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.  

In [ ]:
import geocoder

Loop to add long and lat to df, used arcgis service to find values as it worked more reliably:

In [ ]:
latitude=[]
longitude=[]

for i in df['Postal Code']:
    g = geocoder.arcgis('{},Toronto, Ontario'.format(i))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

df['Latitude'] = pd.DataFrame(latitude)
df['Longitude'] = pd.DataFrame(longitude)

df

## 2.3. Generate maps
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure to add enough Markdown cells to explain what you decided to do and to report any observations you make, and generate maps to visualize your neighborhoods and how they cluster together.

In [ ]:
import folium
from folium import plugins

In [ ]:
# Capture Toronto lat long for use in map
g = geocoder.arcgis('Toronto, Ontario')
lat_lng_coords = g.latlng
latitude=lat_lng_coords[0]
longitude=lat_lng_coords[1]

In [ ]:
# Create folium map
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# Cluster
nb = plugins.MarkerCluster().add_to(toronto_map)

# Add locations to cluster
for lat, lng, label, in zip(df.Latitude, df.Longitude, df.Borough):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(nb)

# Display map
toronto_map

<a id='item3'></a>
# 3. Final Assigment

## 3.1. Description of the problem and a discussion of the background.

## Title: Over and Under

#### Problem:
- High crime regions have less businesses by absolute numbers when compared to low crime regions.

#### Hypothesis:
- The low number of business in high crime regions is due to criminal code which makes certain markets illegal, this includes the sale of prohibited substances. The majority of narcotics charges will be concentrated in high crime regions.
- If the sale or intent to sell of illegal substance is treated as supply, businesses through absolute numbers will have less disparity.
- This will provide insight into any projects, policies or discussions surrounding the adverse affects of stricter control of some substances verses others. Specifically, alcohol verses marijuana.

#### Data:
- This will focus strictly on the city of Chicago, Illinois.
- Data sources:
    1. Chicago crimes, calendar year 2019. ref: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
    2. Foursquare API, current. ref: https://developer.foursquare.com/docs/
    
#### Assumptions:
- Average amount of employees per legal substance business (drinking places, alcoholic beverages) business = 8.2 staff. ref:https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=0&year=2019&qtr=4&own=5&ind=722410&supp=0
- All narcotic charges with intent to sell charges are for the substance marijuana. This is because the data set doesn't differentiate between substances.

#### Future development:
- How would crime driven by illegal markets be affected if these markets were made legal?
- Particular countries and states have made marijuana a legal substance for distribution, how has this affected crime and how has this affected consumption?